In [0]:
grading = False

In [0]:
if not grading: 
  from google.colab import drive
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from collections import defaultdict
import math
import pandas as pd

In [0]:
if not grading:
  # adjust this if you DON'T use google drive but instead upload the files 
  # train_df = pd.read_csv("a7_train.csv")
  train_file_path = "/content/drive/My Drive/a7_train.csv"
  train_df = pd.read_csv(train_file_path)

  train_df
else:
  train_df = pd.read_csv("a7_train.csv")

,class,text
0,neg,I'm sure that the folks on the Texas/Louisiana...
1,neg,The excruciatingly slow pace of this film was ...
2,neg,"I had the ""privilege"" of attending a special s..."
3,neg,I know it's a Power-Rangers gimmick and catere...
4,neg,Lordi was a major hype and revelation in 2007 ...
...,...,...
24995,pos,"As far as I can recall, Balanchine's alteratio..."
24996,pos,"John Van Druten's ""Bell, Book and Candle"" is a..."
24997,pos,This was a great romantic comedy! Historically...
24998,pos,No Strings Attached is one of Carlos Mencia's ...


In [0]:
if not grading:
  # adjust this if you DON'T use google drive but instead upload the files 
  # train_df = pd.read_csv("a7_test.csv")
  test_file_path = "/content/drive/My Drive/a7_test.csv"
  import pandas as pd
  test_df = pd.read_csv(test_file_path)
  test_df
else:
  train_df = pd.read_csv("a7_test.csv")

,class,text
0,neg,Plot Synopsis: Los Angeles in the future. Crim...
1,neg,This movie is something horrible. I was laughi...
2,neg,This film is strictly for fans of Debbie Reyno...
3,neg,This is one the worst movie I've seen and cert...
4,neg,After missing out on this innumerable times on...
...,...,...
24995,pos,"Mark Walhberg in a great role, idolises a rock..."
24996,pos,I too saw this movie when it first came out. I...
24997,pos,"If you haven't seen this yet, you really shoul..."
24998,pos,"This film, also known as ""don't look in the ba..."


In [0]:
def calculate_priors(df):
  """
  TASK 1a)

  Calculates the probability of given classes, based on how often they appear
  in a DataFrame.

  You can assume that the class values for each row are in a column called
  "class".

  !!! Your function must work for any number of classes, so don't assume that 
  the number of classes is always exactly 2! !!!
  
  Return a dictionary mapping the class to its probability.
  Example: The class "apple" appears in two rows, the class "banana" in three.
  The probability for the "apple" class is 2/5, the probability for banana is 
  3/5.
  You have to return a dictionary:
    {"apple": 0.4, "banana": 0.6}

  HINT: You can use .value_counts() to count the values in a column.
    You have to select the correct column first.
  HINT: You can use .apply to modify values with the use of a function, such as
    a lambda function.

  :param df: The DataFrame. 
  :return: A dictionary mapping the class to its probability.
  """
  return {}

In [0]:
if not grading:
  priors = calculate_priors(train_df)

In [0]:
def calculate_class_term_frequency(df, classes):
  """
  TASK 1b)

  For each, calculates the frequency of terms appearing in texts of this class.

  You are given a DataFrame, containing a column "class" and a column "text".
  You are also given a list "classes", containing the possible values for the
  "class" column.
  You can see that each row has an entry for "text" and one for "class". The 
  values in the "text" column are strings (= actual texts).
  Your task is to calculate the frequency (= how often a word appears) for each
  of the classes and individually.

  Example: 
    There are two texts with the "apple" class. 
    There are three texts with the "banana" class.
    The word "fruit" appears five times in the texts of the "apple" class, and 
    two times for texts of the "banana" class.
    The total frequency of the word "fruit" is 7. The frequency for the "apple"
    class is 5. The frequency for the "banana" class is 2.

  To get the individual terms you can use .split(" ") (splitting by whitespace).
  Convert the terms to lowercase.

  HINT: You can iterate over the DataFrame rows with .iterrows():
  https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iterrows.html


  Store the frequencies in a dictionary.
  Return a set containing all the terms and the frequency dictionary.
  
  """
  return set(), {}

In [0]:
if not grading:
  classes = priors.keys()
  terms, freqs_per_class = calculate_class_term_frequency(train_df, classes)

**Conditional Probabilities (with add-1 smoothing)**: $P(w|c) = \frac{count(w, c) + 1}{count(c) + |V|}$

Example: $P(fruit|apple) = 0.3$

$count(w, c)$ = How often word $w$ appears in documents of class $c$.

$count(c)$ = How many words appear in documents of class $c$.

$|V|$ = How many different words exist in the dataset.

**Prior**: $P_{prior}(c) = \frac{N_c}{N}$, where $N$ is the total number of documents and $N_c$ is the number of documents with class $c$

**Choosing a class for document $d$**: $argmax_{c \in C} P(c|d)$ with $P(c|d) = P_{prior}(c) * \prod_{w \in d} P(w|c)$

You can also refer to the lecture slides where you can find an example calculation.


In [0]:
if not grading:
  example_dct = {"fruit": {"apple": 0.3, "banana": 0.4}, "tree": {"apple": 0.5, "banana": 0.1}}
  print(example_dct["fruit"])  # prints the inner dictionary
  # prints the probability of "fruit" appearing in a document if the document has the class "apple"
  print(example_dct["fruit"]["apple"])  

{'apple': 0.3, 'banana': 0.4}
0.3


In [0]:
def calculate_class_term_probs(terms, freqs_per_class):
  """
  TASK 1c)

  Calculates the probability that a term appears in documents of a given class 
  and returns a nested dictionary containing the probabilities.

  Use Laplace (add-1) smoothing.
  Return a nested dictionary. The outer dictionary needs map terms to the 
  inner dictionaries. The inner dictionary maps class labels to probabilities.

  -> If the dictionary is accessed with a non-existing key but an existing class
  it should return the value 1/|V| ! <--

  Example:
    dct = {"fruit": {"apple": 0.3, "banana": 0.4}, "tree": {"apple": 0.5, "banana": 0.1}}
    foo = dct["rock"]["apple"]
    foo == 1/2
    Two terms exist in total (fruit and tree) so the total vocabulary size is 2.


  HINT: Use defaultdict(): https://docs.python.org/3.8/library/collections.html#collections.defaultdict
  """
  return {}

In [0]:
if not grading:
  probs = calculate_class_term_probs(terms, freqs_per_class)

In [0]:
def classify(text, priors, probs):
  """
  TASK 1d)
  This function should predict a class for a given text.
  "text" will always be a single text (string).
  Use the priors you calculated in 1a).
  Use the probabilities you calculated in 1c).
  Use the Naive Bayes classification to assign the class.
  Return the class label as a string.
  """
  return ""

In [0]:
def evaluate(test, priors, probs):
  """
  TASK 1e)
  Evaluate your classification.

  -> You may assume that the only classes are "pos" and "neg" ! <--

  Use the test set for your evaluation.
  For each row in the test set, get the actual class from the "class" column
  and the text from the "text" column.
  Use your "classify(text, priors, probs)" function to get the predicted
  class label for each text.

  HINT: Count the TP, FP, FN for each of the two classes, for each prediction. 

  Calculate the following metrics PER CLASS:
    - precision
    - recall
    - f1-measure
  Return a list of tuples in the following format:
    [(class label, precision, recall, f1-measure), ...]
  """
  return []

In [0]:
if not grading:
  results = evaluate(test_df, priors, probs)